In [0]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

torch.cuda.manual_seed(151515151)
torch.set_default_tensor_type('torch.DoubleTensor')

use_cuda = torch.cuda.is_available()
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [0]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train, )
trainloader = torch.utils.data.DataLoader(trainset, batch_size=10, shuffle=True, **kwargs)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=False, **kwargs)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    print(cm)
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
#         if batch_idx % 100 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    y_test = []
    y_pred = []
    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            y_test += list(target)
            y_pred += list(pred.view_as(target))
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    # Confusion matrix
    confusion_mtx = confusion_matrix(y_test, y_pred)
    plot_confusion_matrix(confusion_mtx, classes=classes, normalize=True,
                          title='Confusion matrix')


In [0]:
class InceptionA(nn.Module):
    def __init__(self, in_channels=384, out_channels=1024):
        super().__init__()
        self.conv1x1_96 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1)
        
        self.conv1x1_64_l2 = nn.Conv2d(in_channels, 64, kernel_size=1, stride=1)
        self.conv3x3_96 = nn.Conv2d(64, out_channels, kernel_size=3, stride=1, padding=1)
        
        self.conv1x1_64_l3 = nn.Conv2d(in_channels, 64, kernel_size=1, stride=1)
        self.conv3x3_96_l3 = nn.Conv2d(64, 96, kernel_size=3, stride=1, padding=1)
        self.conv3x3_96_l3_last = nn.Conv2d(96, out_channels, kernel_size=3, stride=1, padding=1)
        
        self.avg_pool = nn.AvgPool2d(3, stride=1, padding=1, count_include_pad=False)
        self.conv1x1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1)
        
        self.rrelu = nn.LeakyReLU()        
    def forward(self, x):
        
        x0 = self.rrelu(self.conv1x1_96(x))

        x1 = self.rrelu(self.conv1x1_64_l2(x))
        x1 = self.rrelu(self.conv3x3_96(x1))
        
        x2 = self.rrelu(self.conv1x1_64_l3(x))
        x2 = self.rrelu(self.conv3x3_96_l3(x2))
        x2 = self.rrelu(self.conv3x3_96_l3_last(x2))
        
        x3 = self.rrelu(self.avg_pool(x))
        x3 = self.rrelu(self.conv1x1(x3))

        outputs = [x0, x1, x2, x3]
        return torch.cat(outputs, 1)        

In [0]:
class ReductionA(nn.Module):
    def __init__(self, in_channels=384, out_channels=1024):
        super().__init__()
        self.max_pool = nn.MaxPool2d(3, stride=2)
        
        self.conv3x3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2)
        
        self.conv1x1_2 = nn.Conv2d(in_channels, 192, kernel_size=1, stride=1)
        self.conv3x3_2 = nn.Conv2d(192, 224, kernel_size=3, stride=1 , padding=1)
        self.conv3x3_last = nn.Conv2d(224, out_channels, kernel_size=3, stride=2)
        
        self.rrelu = nn.LeakyReLU()
    def forward(self, x):
        x0 = self.rrelu(self.conv3x3_1(x))
        x1 = self.rrelu(self.conv1x1_2(x))
        x1 = self.rrelu(self.conv3x3_2(x1))
        x1 = self.rrelu(self.conv3x3_last(x1))
        x2 = self.rrelu(self.max_pool(x))
        outputs = [x0, x1, x2]
        return torch.cat(outputs, 1)        

In [0]:
class Net(nn.Module):
    """Experiment relu and linear layer with as is stem"""
    def __init__(self):
        super().__init__()
        self.inception_a = InceptionA(in_channels=3)
        self.reduction_a = ReductionA(in_channels=4096, out_channels=384)
        self.mp = nn.MaxPool2d(2)
        self.rrelu = nn.LeakyReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.fc_last = nn.Linear(43776, 10)

    def forward(self, x):
        in_size = x.size(0)
        
        x = self.rrelu(self.mp(self.inception_a(x)))
        x = self.rrelu(self.mp(self.reduction_a(x)))
        
        x = x.view(in_size, -1) # flatten the tensor
        
        x = F.rrelu(self.fc_last(x))
        x = self.dropout(x)
        return F.log_softmax(x, dim=1)

In [0]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

In [0]:
device = 'cuda'

In [0]:
model = Net().to(device)
model = model.cuda()
model.apply(init_weights)
model = model.float()

In [0]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [0]:
for epoch in range(1, 1 + 1):
    train(model, 'cuda', trainloader, optimizer, epoch)
    test(model, 'cuda', testloader)

In [0]:
model.cuda